In [9]:
import matplotlib.pyplot as plt
from utils import CrowdsDataset
import torch
from torchsummary import summary
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from utils import train_transform, val_transform
from models import CGDRCN1, MyFC
import os
from utils import calculate_mae, compute_errors
from config import learning_rate, batch_size, num_epochs, momentum, norm, save_path, folder_path, image_data_train_path, image_data_val_path

In [10]:
#device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [11]:
"""learning_rate= 0.000001
batch_size = 24
num_epochs = 50
momentum = 0.9
norm = 100
save_path = './models/model.pt'"""

"learning_rate= 0.000001\nbatch_size = 24\nnum_epochs = 50\nmomentum = 0.9\nnorm = 100\nsave_path = './models/model.pt'"

In [12]:
"""folder_path = './processed_data'

gt_data_train_path = './processed_data/train/gt'
image_data_train_path = './processed_data/train/img'
density_data_train_path = './processed_data/train/den'

gt_data_val_path = './processed_data/val/gt'
image_data_val_path = './processed_data/val/img'
density_data_val_path = './processed_data/val/den'

gt_data_test_path = './processed_data/test/gt'
image_data_test_path = './processed_data/test/img'
density_data_test_path = './processed_data/test/den'"""

"folder_path = './processed_data'\n\ngt_data_train_path = './processed_data/train/gt'\nimage_data_train_path = './processed_data/train/img'\ndensity_data_train_path = './processed_data/train/den'\n\ngt_data_val_path = './processed_data/val/gt'\nimage_data_val_path = './processed_data/val/img'\ndensity_data_val_path = './processed_data/val/den'\n\ngt_data_test_path = './processed_data/test/gt'\nimage_data_test_path = './processed_data/test/img'\ndensity_data_test_path = './processed_data/test/den'"

In [13]:
crowd_dataset = CrowdsDataset(path = image_data_train_path.rsplit('/',1)[0], transform=train_transform)
crowd_dataset_val = CrowdsDataset(path = image_data_val_path.rsplit('/',1)[0], transform=val_transform)


In [14]:
train_loader =  DataLoader(crowd_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(crowd_dataset_val, batch_size=batch_size, shuffle=True, num_workers=0)

In [15]:
model = MyFC(scale_factor=32)
vgg = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

for i, layer in enumerate(vgg.features):
    if isinstance(layer, torch.nn.Linear):
        break

print(i)
model2 = torch.nn.Sequential(*list(vgg.features.children())[:i])
model2 = CGDRCN1(model2)
#model = model2
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)

30
MyFC(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [16]:
summary(model,(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
         MaxPool2d-5         [-1, 64, 128, 128]               0
            Conv2d-6        [-1, 128, 128, 128]          73,856
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]         147,584
              ReLU-9        [-1, 128, 128, 128]               0
        MaxPool2d-10          [-1, 128, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]         295,168
             ReLU-12          [-1, 256, 64, 64]               0
           Conv2d-13          [-1, 256, 64, 64]         590,080
             ReLU-14          [-1, 256,

In [17]:
min_mae = 10000
min_mae_epoch = -1
train_losses = []
maes = []
for epoch in range(num_epochs):
  model.train()
  epoch_loss = 0.0
  loop = tqdm(train_loader, leave=True)
  for i, data in enumerate(loop):
    image = data['image'].to(device)
    den = data['den'].to(device)
    optimizer.zero_grad()
    outputs = model(image)
    loss = criterion(outputs,den)
    epoch_loss += loss.item()
    loss.backward()
    optimizer.step()
  train_losses.append(epoch_loss/len(train_loader))
  print('epoch_loss: ', epoch_loss/len(train_loader))
  model.eval()
  with torch.no_grad():
    calculate_mae(model, device, image_data_val_path,'predictions', 'val', norm)
    errors = compute_errors(folder_path, 'predictions', 'val')
    epoch_mae = int(float(errors[0]))
    maes.append(int(float(errors[0])))

  if os.path.exists('predictions'):
    os.remove('predictions')

  if epoch_mae < min_mae:
    min_mae, min_mae_epoch = epoch_mae, epoch
    torch.save(model, save_path)     # save checkpoints
  print('Epoch ', epoch, ' MAE: ', epoch_mae, 'Min MAE: ', min_mae, ' Min Epoch: ', min_mae_epoch)   # print information


# plot the train losses and MAEs
maes = [x/10000 for x in maes]
fig, ax = plt.subplots()
ax.plot(range(num_epochs), train_losses, label='train loss')
ax.plot(range(num_epochs), maes, label='validation MAE')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss/MAE')
ax.set_title('Training Progress')
ax.legend()
plt.show()

  1%|          | 1/95 [00:34<53:27, 34.13s/it]


KeyboardInterrupt: 